In [9]:
# Set up libraries and backend
from qiskit.tools.jupyter import * 
from qiskit import IBMQ
from qiskit import assemble
import numpy as np
import matplotlib.pyplot as plt
from qiskit import pulse
from qiskit.pulse import pulse_lib
from qiskit.pulse import Play
from qiskit.pulse.commands import SamplePulse
from qiskit.tools.monitor import job_monitor
from scipy.optimize import curve_fit
import json

%config InlineBackend.figure_format = 'svg'
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')
backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support OpenPulse"

# Use for conversions later
GHz = 1.0e9
MHz = 1.0e6
kHz = 1.0e3
ms = 1.0e-3
us = 1.0e-6
ns = 1.0e-9
scale_factor = 1e-14

# Initialize qubit to |0> and retrieve backend frequencies
qubit = 0
dt = backend_config.dt
backend_defaults = backend.defaults()
qubit_props_dict = backend.properties().qubit_property(0)
rough_qubit_freq = qubit_props_dict['frequency'][0]
rough_cav_freq = backend_defaults.meas_freq_est[qubit]


# Define and configure measurement map (hardware constraint)
meas_map_idx = None
for i, measure_group in enumerate(backend_config.meas_map):
    if qubit in measure_group:
        meas_map_idx = i
        break
assert meas_map_idx is not None, f"Couldn't find qubit {qubit} in meas_map!"
drive_chan = pulse.DriveChannel(qubit)
meas_chan = pulse.MeasureChannel(qubit)
acq_chan = pulse.AcquireChannel(qubit)
qubit_meas_group = backend_config.meas_map[meas_map_idx]
inst_sched_map = backend_defaults.instruction_schedule_map
measure = inst_sched_map.get('measure', qubits=qubit_meas_group)


# Define for convenience
def get_closest_multiple_of_16(num):
     return int(num + 8) - (int(num + 8) % 16)

# Functions to save data to and load data from external text files
def saveData(dataset, file):
    filehandler = open(file, 'w')
    json.dump(dataset, filehandler)
    filehandler.close()
    
def loadData(file):
    filehandler = open(file)
    data = json.load(filehandler)
    filehandler.close()
    return data

ibmqfactory.load_account:WARNING:2020-10-19 20:16:17,532: Credentials are already in use. The existing account in the session will be replaced.


In [10]:
# Get cavity frequency based on calibration experiment
cavity_data = loadData('CavityParameters.txt')
cav_freq_Hz = cavity_data['wc']
cav_freq_GHz = cav_freq_Hz / GHz

In [11]:
# Measurement + acquisition parameters
meas_amp = 0.3
acq_samp_us = 0.05
meas_sigma_us = 0.01
meas_risefall_us = 0.01

acq_samp = get_closest_multiple_of_16(acq_samp_us * us/dt)
meas_sigma = get_closest_multiple_of_16(meas_sigma_us * us/dt)
meas_risefall = get_closest_multiple_of_16(meas_risefall_us * us/dt)

meas_pulse = pulse_lib.gaussian_square(amp=meas_amp, duration=acq_samp, sigma=meas_sigma, risefall=meas_risefall)
acq_pulse = pulse.Acquire(duration=acq_samp, channel=[pulse.AcquireChannel(i) for i in qubit_meas_group],
                         mem_slot=[pulse.MemorySlot(i) for i in qubit_meas_group])

# Create schedule for measurement
meas_schedule = pulse.Schedule(name='Measurement schedule')
meas_schedule += Play(meas_pulse, meas_chan)
meas_schedule += acq_pulse

# Split cavity drives into 2 diff jobs in order to maximize drive amplitude without exceeding memory restrictions

# Lower cavity drive job
cav_dur_min_us_0 = 1.0
cav_dur_max_us_0 = 9.0
num_durs_0 = 10

# Higher cavity drive job (less data points allowed since higher drive times --> greater memory per pulse)
cav_dur_min_us_1 = 9.0
cav_dur_max_us_1 = 13.0
num_durs_1 = 5

# Maximal amplitude --> adjusting this would allow for adjustment of duration ranges above
cav_amp = 1.0

cav_durs_us_0 = np.linspace(cav_dur_min_us_0, cav_dur_max_us_0, num_durs_0, endpoint=True)
cav_durs_us_1 = np.linspace(cav_dur_min_us_1, cav_dur_max_us_1, num_durs_1, endpoint=True)
cav_durs_0 = [get_closest_multiple_of_16(dur_us * us/dt) for dur_us in cav_durs_us_0]
cav_durs_1 = [get_closest_multiple_of_16(dur_us * us/dt) for dur_us in cav_durs_us_1]

In [12]:
# Get pi pulse parameters from calibration experiment, construct pulse

pi_data = loadData('pipulse.txt') # from calibration experiment
drive_samps_us = pi_data['drive_samps_us'] # Width of the gaussian pulse
drive_sigma_us = pi_data['drive_sigma_us'] # Truncates duration of gaussian to be finite
pi_amp = pi_data['pi_amp']

drive_samps = get_closest_multiple_of_16(drive_samps_us * us/dt) # Puts width in units of dt
drive_sigma = get_closest_multiple_of_16(drive_sigma_us * us/dt) # Puts duration in units of dt

pi_pulse = pulse_lib.gaussian(amp=pi_amp, sigma=drive_sigma, duration=drive_samps)

In [13]:
# Ground and excited schedules where each has different location of acquisition during drive

schedules_gnd_0 = []
for dur in cav_durs_0:
    cav_pulse = pulse_lib.gaussian_square(amp=cav_amp, duration=dur, sigma=meas_sigma, risefall=meas_risefall)
    schedule = pulse.Schedule(name=f"Cav duration {dur} (dt)")
    schedule += Play(cav_pulse, meas_chan)
    schedule += meas_schedule
    schedules_gnd_0.append(schedule)
    
schedules_gnd_1 = []
for dur in cav_durs_1:
    cav_pulse = pulse_lib.gaussian_square(amp=cav_amp, duration=dur, sigma=meas_sigma, risefall=meas_risefall)
    schedule = pulse.Schedule(name=f"Cav duration {dur} (dt)")
    schedule += Play(cav_pulse, meas_chan)
    schedule += meas_schedule
    schedules_gnd_1.append(schedule)
    
schedules_exc_0 = []
for dur in cav_durs_0:
    cav_pulse = pulse_lib.gaussian_square(amp=cav_amp, duration=dur, sigma=meas_sigma, risefall=meas_risefall)
    schedule = pulse.Schedule(name=f"Cav duration {dur} (dt)")
    schedule += Play(pi_pulse, drive_chan)
    schedule += Play(cav_pulse, meas_chan) << schedule.duration
    schedule += meas_schedule
    schedules_exc_0.append(schedule)
    
schedules_exc_1 = []
for dur in cav_durs_1:
    cav_pulse = pulse_lib.gaussian_square(amp=cav_amp, duration=dur, sigma=meas_sigma, risefall=meas_risefall)
    schedule = pulse.Schedule(name=f"Cav duration {dur} (dt)")
    schedule += Play(pi_pulse, drive_chan)
    schedule += Play(cav_pulse, meas_chan) << schedule.duration
    schedule += meas_schedule
    schedules_exc_1.append(schedule)

In [14]:
# Number of shots performed in each Qobj
num_shots = 1024

program_gnd_0 = assemble(schedules_gnd_0, backend=backend, shots=num_shots,
                       meas_level=1, meas_return='avg', schedule_los=[{meas_chan: cav_freq_Hz}]*len(schedules_gnd_0))

program_gnd_1 = assemble(schedules_gnd_1, backend=backend, shots=num_shots,
                       meas_level=1, meas_return='avg', schedule_los=[{meas_chan: cav_freq_Hz}]*len(schedules_gnd_1))
                       
program_exc_0 = assemble(schedules_exc_0, backend=backend, shots=num_shots,
                      meas_level=1, meas_return='avg', schedule_los=[{meas_chan: cav_freq_Hz}]*len(schedules_exc_0))

program_exc_1 = assemble(schedules_exc_1, backend=backend, shots=num_shots,
                      meas_level=1, meas_return='avg', schedule_los=[{meas_chan: cav_freq_Hz}]*len(schedules_exc_1))

job_gnd_0 = backend.run(program_gnd_0)
jobID_gnd_0 = job_gnd_0.job_id()
job_monitor(job_gnd_0)
print(f"Job gnd 0 error: {job_gnd_0.error_message()}")

job_gnd_1 = backend.run(program_gnd_1)
jobID_gnd_1 = job_gnd_1.job_id()
job_monitor(job_gnd_1)
print(f"Job gnd 1 error: {job_gnd_1.error_message()}")

job_exc_0 = backend.run(program_exc_0)
jobID_exc_0 = job_exc_0.job_id()
job_monitor(job_exc_0)
print(f"Job exc 0 error: {job_exc_0.error_message()}")

job_exc_1 = backend.run(program_exc_1)
jobID_exc_1 = job_exc_1.job_id()
job_monitor(job_exc_1)
print(f"Job exc 1 error: {job_exc_1.error_message()}")

Job Status: job incurred error     
Job gnd 0 error: Internal Error. Error code: 9999.
Job Status: job incurred error     
Job gnd 1 error: Internal Error. Error code: 9999.
Job Status: job incurred error     
Job exc 0 error: Internal Error. Error code: 9999.
Job Status: job incurred error     
Job exc 1 error: Internal Error. Error code: 9999.


'Internal Error. Error code: 9999.'

In [16]:
# Function for quickly extracting results from jobs
def get_job_data(job):
    result = job.result(timeout=120)
    sweep = []
    for i in range(len(result.results)):
        res = result.get_memory(i)*scale_factor
        sweep.append(res[qubit])
    return sweep

# Retrieve data
data_gnd_0 = get_job_data(job_gnd_0)
data_gnd_1 = get_job_data(job_gnd_1)
data_exc_0 = get_job_data(job_exc_0)
data_exc_1 = get_job_data(job_exc_1)

# Datasets in diff jobs are off by diff phases --> 
# use overlap data point (@dur = 9.0 us) in gnd and exc datasets to shift accordingly for each pair
phase_gnd = data_gnd_1[0] - data_gnd_0[-1]
phase_exc = data_exc_1[0] - data_exc_0[-1]
shifted_data_gnd_1 = data_gnd_1 - phase_gnd
shifted_data_exc_1 = data_exc_1 - phase_exc

# Concatenate datasets to use as whole
data_gnd = data_gnd_0 + shifted_data_gnd_1
data_exc = data_exc_0 + shifted_data_exc_1
cav_durs_us = np.concatenate((cav_durs_us_0, cav_durs_us_1))

# Plot evolution of data over total time cavity driven
plt.scatter(cav_durs_us, np.abs(data_gnd), color='black')
plt.scatter(cav_durs_us, np.real(data_gnd), color='blue')
plt.scatter(cav_durs_us, np.imag(data_gnd), color='red')
plt.xlabel("Cavity drive time [us]")
plt.ylabel("Quadrature [au]")
plt.title("Ground State Stabilization over Time Driven")
plt.show()

plt.scatter(cav_durs_us, np.abs(data_exc), color='black')
plt.scatter(cav_durs_us, np.real(data_exc), color='blue')
plt.scatter(cav_durs_us, np.imag(data_exc), color='red')
plt.xlabel("Cavity drive time [us]")
plt.ylabel("Quadrature [au]")
plt.title("Excited State Stabilization over Time Driven")
plt.show()

IBMQJobFailureError: 'Unable to retrieve result for job 5f8e2c56dc47090013ed1b1b. Job has failed. Use job.error_message() to get more details.'

In [ ]:
# Plot evolution of quadrature components over time (black is first point in time, red is last)

plt.scatter(np.real(data_gnd), np.imag(data_gnd), color='blue')
plt.scatter(np.real(data_gnd[0]), np.imag(data_gnd[0]), color='black')
plt.scatter(np.real(data_gnd[-1]), np.imag(data_gnd[-1]), color='red')
plt.xlabel("I [au]")
plt.ylabel("Q [au]")
plt.title("Quadratures of Ground State")
plt.show()

plt.scatter(np.real(data_exc), np.imag(data_exc), color='blue')
plt.scatter(np.real(data_exc[0]), np.imag(data_exc[0]), color='black')
plt.scatter(np.real(data_exc[-1]), np.imag(data_exc[-1]), color='red')
plt.xlabel("I [au]")
plt.ylabel("Q [au]")
plt.title("Quadratures of Excited State")
plt.show()